In [65]:
import pandas as pd
import requests as r

from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

# https://cronos.org/explorer/api-docs#stats
# base takes data in yyyy-mm-dd
api_base = 'https://cronos.org/explorer/api?module=stats&action=totalfees&date='




In [66]:
trailing_num_days = 120
start_date = date.today()-timedelta(days=trailing_num_days)
print(start_date)

2022-07-12


In [67]:
arr = []
num_runs = 0
for i in (start_date + timedelta(n) for n in range(0,trailing_num_days)):
        try:
                res = r.get(api_base+str(i)).json()
                fees = int(res['result'])/10**18
                # res = int(res) / 10**18
                if num_runs == 0:
                        arr = [[i,fees]]
                else:
                        arr.append([i,fees])
                # print([i,res['result']])
                num_runs += 1
        except:
                continue
        time.sleep(1)
df = pd.DataFrame(arr,columns=['date','native_fees'])


,date,native_fees
0,2022-07-12,102129.172084
1,2022-07-13,109371.944655
2,2022-07-14,112582.392871
3,2022-07-15,107440.810064
4,2022-07-16,101635.055778
...,...,...
115,2022-11-04,97278.864396
116,2022-11-05,93826.462763
117,2022-11-06,104925.008254
118,2022-11-07,94810.016418


In [87]:
df['date'] = df['date'].astype('datetime64[ns]')
display(df)

,date,native_fees
0,2022-07-12,102129.172084
1,2022-07-13,109371.944655
2,2022-07-14,112582.392871
3,2022-07-15,107440.810064
4,2022-07-16,101635.055778
...,...,...
115,2022-11-04,97278.864396
116,2022-11-05,93826.462763
117,2022-11-06,104925.008254
118,2022-11-07,94810.016418


In [77]:
cgid = 'crypto-com-chain'
cg_api = 'https://api.coingecko.com/api/v3/coins/@id@/market_chart?vs_currency=usd&days=@days@&interval=daily'
cg_api = cg_api.replace('@id@',cgid)
cg_api = cg_api.replace('@days@',str(trailing_num_days+1))
prices = r.get(cg_api,headers=header).json()['prices']
pr_df = pd.DataFrame(prices,columns=['date','price_usd'])
pr_df['date'] = pd.to_datetime(pr_df['date'],unit = 'ms')
## OFFSET BY 1 SO THIS BECOMES END OF DAY PRICE
pr_df['date'] = pr_df['date'] - pd.Timedelta(days=1)
pr_df.drop(pr_df.tail(1).index,inplace=True)

          date  price_usd
0   2022-07-11   0.113302
1   2022-07-12   0.110496
2   2022-07-13   0.115898
3   2022-07-14   0.118417
4   2022-07-15   0.119860
..         ...        ...
116 2022-11-04   0.117023
117 2022-11-05   0.121254
118 2022-11-06   0.118380
119 2022-11-07   0.124900
120 2022-11-08   0.103417

[121 rows x 2 columns]


In [91]:
final_df = df.merge(pr_df,on='date',how='left')
final_df['fees_usd'] = final_df['native_fees'] * final_df['price_usd']
display(final_df)
final_df.to_csv('cronos_fees.csv')

,date,native_fees,price_usd,fees_usd
0,2022-07-12,102129.172084,0.110496,11284.872836
1,2022-07-13,109371.944655,0.115898,12675.942261
2,2022-07-14,112582.392871,0.118417,13331.625464
3,2022-07-15,107440.810064,0.119860,12877.844768
4,2022-07-16,101635.055778,0.122626,12463.110432
...,...,...,...,...
115,2022-11-04,97278.864396,0.117023,11383.882822
116,2022-11-05,93826.462763,0.121254,11376.833396
117,2022-11-06,104925.008254,0.118380,12421.060480
118,2022-11-07,94810.016418,0.124900,11841.726668
